In [1]:
import os
import sys
import glob
import datetime as dt
import pandas as pd
import numpy as np
import blmfire as blm
from importlib import reload
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
testfile_current = "./testdata/blmfiredata.txt"
testfile_prev = "./testdata/20170713_065905_blmfiredata.txt"

In [3]:
fieldmap = blm.FIELDMAP
fieldmap['lastupdatedatetime'] = 'firesadmin.dailyfirerecord.lastupdatedatetime'
fieldmap['discoverydatetime'] = 'firesadmin.fire.discoverydatetime'
fieldmap['guid'] = 'firesadmin.fire.irwinid'

In [4]:
def get_data(fn):
    data = blm.get_raw_fire_data(fn, tryremote=False)
    fieldnames = blm.sanitize_dataline(data[0], blanks=3, lower=True)
    rawdata = [blm.sanitize_dataline(
                item.strip(), blanks=2, lower=False) for item in data[1::2]]
    
    # the transformed data will be a list of dictionaries
    data = blm.get_firedata(fieldnames, rawdata)
    return fieldnames, data

In [5]:
fieldnames, data = get_data(testfile_current)

In [6]:
_, data_prev = get_data(testfile_prev)

In [7]:
def cleanup_df(dataframe):
    dataframe['discoverydatetime'] = pd.to_datetime(dataframe['discoverydatetime'], unit='ms')
    dataframe['lastupdatedatetime'] = pd.to_datetime(dataframe['lastupdatedatetime'], unit='ms')
    dataframe['containmentdatetime'] = pd.to_datetime(dataframe['containmentdatetime'], unit='ms')
    try:
        dataframe['controldatetime'] = pd.to_datetime(dataframe['controldatetime'], unit='ms')
    except KeyError:  # can be empty
        pass
    dataframe['acreage'] = pd.to_numeric(dataframe['acreage'])
    return dataframe

In [9]:
df_current = pd.DataFrame(data)[list(fieldmap.values())]
df_current.columns = [list(fieldmap.keys())]
df_prev = pd.DataFrame(data_prev)[list(fieldmap.values())]
df_prev.columns = [list(fieldmap.keys())]

In [10]:
df_current = cleanup_df(df_current)
df_prev = cleanup_df(df_prev)

KeyError: 'containmentdatetime'

In [18]:
df_prev[~df_prev['designation'].isin(df_current['designation'])].to_csv()

',cause,longitude,guid,structuresburned,acreage,maintext,designation,discoverydatetime,latitude,primaryfueltype,numberpeople,structuresthreatened,lastupdatedatetime\n36,Lightning,-153.4959444,8976669B-225E-490A-B614-1C4177A29BCB,0,19.6,Zone aircraft N309VS flew the fire area today. Fire personnel reported no change in size or fire behavior. Update in acreage due to more accurate mapping.,North Kusko,2017-06-16 17:25:11.529,63.6799445,Black Spruce,0,0,2017-06-20 12:48:00.529\n55,Lightning,-141.9646944,D5E3FDDD-7110-4B2E-BFDD-309649EA21F1,0,3.1,Landsat imagery confirmed initial coordinates and fire size.,Martin Creek,2017-07-02 17:15:08.544,68.0495556,Tundra,0,0,2017-07-08 11:51:00.544\n56,Lightning,-141.9171667,D078470D-8A60-480F-8647-45B43214A593,0,0.1,The fire was flown by zone aircraft N9011N. No fire activity was observed.,Coleen River,2017-07-02 17:18:46.544,68.0861667,Tundra,0,0,2017-07-09 18:56:00.544\n'

In [19]:
df_current[~df_current['designation'].isin(df_prev['designation'])]

,cause,longitude,guid,structuresburned,acreage,maintext,designation,discoverydatetime,latitude,primaryfueltype,numberpeople,structuresthreatened,lastupdatedatetime
46,Lightning,-144.6955833,C6ED6EFA-EBCF-4B58-8AAA-260BEBBDA871,0,1.0,The fire was discovered by zone aircraft N222M...,WL 543T,2017-07-13 17:15:04.730,67.0030278,Black Spruce,0,0,2017-07-13 19:52:00.730
47,Lightning,-144.316667,607BB758-6C06-4E35-8A4A-9B5EFEF53D52,0,0.0,A small column of smoke was reported to the du...,George Lake Hills,2017-07-13 08:10:24.731,63.75,Unknown,0,0,2017-07-14 04:03:45.731


In [20]:
today = dt.datetime.now()
df_current.loc[(today - df_current['lastupdatedatetime'] < dt.timedelta(days=2))]

,cause,longitude,guid,structuresburned,acreage,maintext,designation,discoverydatetime,latitude,primaryfueltype,numberpeople,structuresthreatened,lastupdatedatetime


In [21]:
df_prev.set_index('guid')[['lastupdatedatetime', 'acreage']]

,lastupdatedatetime,acreage
guid,,
231B0E11-EA79-4FFD-A44C-84130064B858,2017-07-09 09:17:00.513,178.2
8719131D-F468-481F-A61A-8DF15C13887B,2017-07-04 10:40:00.513,4.7
3CE4F630-EF46-49EF-9A2D-3186E3DCE0C7,2017-07-08 22:05:00.513,20.0
E1963C4B-0888-4A60-B5B8-D8FFA8B8DF3A,2017-07-12 20:49:00.513,356.8
B18CE0AB-470E-49A5-97B5-3B1790C668C9,2017-07-04 22:11:00.513,332.0
5F36D2F7-347D-4FBA-8C2C-0B21984A7607,2017-07-07 17:51:00.513,25.0
1E5D3F64-62ED-4149-AB71-53535BDD0504,2017-07-12 20:46:00.513,22227.3
8FC76EFB-5229-42B5-8ED8-D1F9B325106A,2017-07-12 17:43:00.513,4.0
21207F32-B0BD-42F8-A1FA-C4A7AC6EC464,2017-07-11 06:31:00.513,235.4


In [22]:
df_joined = df_current.join(df_prev.set_index('guid')[['lastupdatedatetime', 'acreage', 'maintext']], on='guid', rsuffix='_prev')

In [23]:
df_joined['delta_acres'] = df_joined['acreage'] - df_joined['acreage_prev']

In [24]:
df_joined.iloc[[6, 9,12, 22, 23, 27, 47, 49, 51, 55]][['designation', 'latitude', 'longitude', 'delta_acres']].to_csv('testout')

In [25]:
df_current.sort_values('acreage', ascending=False, axis=0).head()

,cause,longitude,guid,structuresburned,acreage,maintext,designation,discoverydatetime,latitude,primaryfueltype,numberpeople,structuresthreatened,lastupdatedatetime
39,Lightning,-141.3181389,CB846635-C8CB-4734-8DC9-2C787C6C58F6,0,62800.9,The fire was flown by surveillance aircraft N2...,White Mountain Creek,2017-07-03 14:09:32.724,67.4532778,Black Spruce,0,0,2017-07-14 09:48:00.724
23,Lightning,-141.0150278,C1446D0C-39BF-46C9-BD52-E709BD0B7902,0,45521.7,At 1500 the fire was flown by zone surveillanc...,Campbell River,2017-06-26 17:14:24.711,67.2183056,Black Spruce,35,0,2017-07-13 21:05:00.711
49,Lightning,-142.0123333,022B3833-8BAA-4CF8-AC50-CF235F065649,0,27629.4,Zone aircraft N222ME flew the fire and reporte...,Lois Creek,2017-07-02 17:28:36.733,68.2916667,Tussock,0,0,2017-07-13 18:40:00.733
12,Lightning,-141.3226944,86A07501-8395-4D5D-B812-A18DA398F01D,0,23673.6,Zone aircraft N222ME flew the fire and reporte...,Ammerman Creek,2017-06-23 17:05:48.701,68.4175833,Black Spruce,0,0,2017-07-13 18:51:00.701
27,Lightning,-143.7333333,DC5D8B8C-8437-4C56-8822-187E33619168,0,23383.9,Zone surveillance aircraft N222ME flew the fir...,Boulder Creek,2017-07-02 15:12:31.714,67.45,Black Spruce,75,0,2017-07-14 09:49:00.714


In [26]:
condition_recent = (today - df_current['lastupdatedatetime'] < dt.timedelta(days=2))

In [156]:
df_current[condition_recent]['maintext'].to_csv()

'2,Zone surveillance aircraft N309VS flew the fire and reported no visible smoke.\n3,"On 7/13, the fire received precipitation and no dominant smokes were visible from the highway; however, drift smoke was present in the area. An unmanned UAS flew the fire and captured infrared images indicating areas of heat. On 7/14, all resources will demobilize from the fire and the fire will remain in monitor status. Fire size decreased to 332 acres due to more accurate mapping."\n6,Zone surveillance aircraft N222ME flew the area of the fire but was unable to observe it due to heavy smoke.\n7,The fire was demobilized and placed into monitor status.\n9,Zone surveillance aircraft N222ME flew the fire and reported it with a 10 percent active perimeter. The fire behavior was described as smoldering in a spruce/hardwood/brush mix. Observations were taken under 40 percent cloud cover.\n11,"The fire was flown by zone aircraft N22ME. No smoke, growth or active perimeter was reported."\n12,"Zone aircraft N

In [160]:
count_vect = CountVectorizer()
firetextcounts = count_vect.fit_transform(df_current[condition_recent]['maintext'])

In [163]:
count_vect.vocabulary_.get('no smoke')

In [166]:
print(df_current[condition_recent]['maintext'].to_csv())

2,Zone surveillance aircraft N309VS flew the fire and reported no visible smoke.
3,"On 7/13, the fire received precipitation and no dominant smokes were visible from the highway; however, drift smoke was present in the area. An unmanned UAS flew the fire and captured infrared images indicating areas of heat. On 7/14, all resources will demobilize from the fire and the fire will remain in monitor status. Fire size decreased to 332 acres due to more accurate mapping."
6,Zone surveillance aircraft N222ME flew the area of the fire but was unable to observe it due to heavy smoke.
7,The fire was demobilized and placed into monitor status.
9,Zone surveillance aircraft N222ME flew the fire and reported it with a 10 percent active perimeter. The fire behavior was described as smoldering in a spruce/hardwood/brush mix. Observations were taken under 40 percent cloud cover.
11,"The fire was flown by zone aircraft N22ME. No smoke, growth or active perimeter was reported."
12,"Zone aircraft N222ME f

In [167]:
df_current.to_csv('current.csv', sep='\t')

In [156]:
reload(blm)

<module 'blmfire' from '/Users/chris/Dropbox/Coding/Python/aiccfiredata/blmfire.py'>

In [37]:
import pandas as pd

## Test JavaScript generation 

In [31]:
FIRETEMPLATE = 'templates/firetemplate.js'
HTMLTEMPLATE = 'templates/current_fires_template.js'

In [46]:
data = blm.get_raw_fire_data(testfile_current, tryremote=False)
fieldnames = blm.sanitize_dataline(data[0], blanks=3, lower=True)
rawdata = [blm.sanitize_dataline(
            item.strip(), blanks=2, lower=False) for item in data[1::2]]

# the transformed data will be a list of dictionaries
data = blm.get_firedata(fieldnames, rawdata)
finalfirelist = blm.select_relevant_values(data)

In [47]:
finalfirelist

[{'acreage': '178.2',
  'cause': 'Lightning',
  'current': 'false',
  'designation': 'North Hills',
  'discovered': '08 Jun 2017, 06:00',
  'lastupdated': '09 Jul 2017, 01:17',
  'latitude': '62.1606667',
  'longitude': '-163.1690833',
  'maintext': 'Decrease in acreage with updated perimeter.',
  'numberpeople': '0',
  'primaryfueltype': 'Tundra',
  'structuresburned': '0',
  'structuresthreatened': '0'},
 {'acreage': '4.7',
  'cause': 'Lightning',
  'current': 'false',
  'designation': 'Kowlak',
  'discovered': '28 Jun 2017, 09:36',
  'lastupdated': '04 Jul 2017, 02:47',
  'latitude': '69.9508333',
  'longitude': '-157.8397222',
  'maintext': 'Decrease in acreage due to new LandSat perimeter.',
  'numberpeople': '0',
  'primaryfueltype': 'Tundra',
  'structuresburned': '0',
  'structuresthreatened': '0'},
 {'acreage': '20',
  'cause': 'Lightning',
  'current': 'true',
  'designation': 'Garnet Creek',
  'discovered': '06 Jul 2017, 08:29',
  'lastupdated': '13 Jul 2017, 11:25',
  'lati

In [49]:
blm.PROJECTPATH = '.'

In [50]:
fires = blm.fill_template(finalfirelist, template=FIRETEMPLATE)


In [52]:
print(fires)

	fire = {
		designation: "North Hills",
		center: new google.maps.LatLng(62.1606667, -163.1690833),
		acreage: 178.2,
		maintext: "Decrease in acreage with updated perimeter.",
		lastupdated: "09 Jul 2017, 01:17",
		discovered: "08 Jun 2017, 06:00",
		current: false
	};
	firemap.push(fire);
	fire = {
		designation: "Kowlak",
		center: new google.maps.LatLng(69.9508333, -157.8397222),
		acreage: 4.7,
		maintext: "Decrease in acreage due to new LandSat perimeter.",
		lastupdated: "04 Jul 2017, 02:47",
		discovered: "28 Jun 2017, 09:36",
		current: false
	};
	firemap.push(fire);
	fire = {
		designation: "Garnet Creek",
		center: new google.maps.LatLng(65.3315, -150.6751667),
		acreage: 20,
		maintext: "Zone surveillance aircraft N309VS flew the fire and reported no visible smoke.",
		lastupdated: "13 Jul 2017, 11:25",
		discovered: "06 Jul 2017, 08:29",
		current: true
	};
	firemap.push(fire);
	fire = {
		designation: "Dietrich River",
		center: new google.maps.LatLng(67.8851389, -149.807

## Globbedy-glob glob glob ...

In [8]:
ALREADYLOADED = 'aiccfilelist.txt'
LASTFIREDATA = 'firedata.pickle'

In [9]:
try:
    with open(ALREADYLOADED, 'r') as source:
        oldfilelist = [
            fn.strip() for fn in source.readlines()
        ]
except IOError:
    oldfilelist = []
filelist = sorted(
    set(glob.glob('archive/2017*_blmfiredata.txt')) - set(oldfilelist)
)

In [10]:
for file in filelist:
    print(file)

archive/20170726_025909_blmfiredata.txt
archive/20170726_035911_blmfiredata.txt
archive/20170726_045910_blmfiredata.txt
archive/20170726_055909_blmfiredata.txt
archive/20170726_065912_blmfiredata.txt
archive/20170726_135908_blmfiredata.txt
archive/20170726_145909_blmfiredata.txt
archive/20170726_155908_blmfiredata.txt
archive/20170726_165911_blmfiredata.txt
archive/20170726_175909_blmfiredata.txt
archive/20170726_185907_blmfiredata.txt
archive/20170726_195906_blmfiredata.txt
archive/20170726_205906_blmfiredata.txt
archive/20170726_215905_blmfiredata.txt
archive/20170726_225906_blmfiredata.txt
archive/20170726_235907_blmfiredata.txt
archive/20170727_005907_blmfiredata.txt
archive/20170727_015907_blmfiredata.txt


In [11]:
fieldmap = blm.FIELDMAP
fieldmap['lastupdatedatetime'] = 'firesadmin.dailyfirerecord.lastupdatedatetime'
fieldmap['discoverydatetime'] = 'firesadmin.fire.discoverydatetime'
fieldmap['controldatetime'] = 'firesadmin.fire.controldatetime'
fieldmap['containmentdatetime'] = 'firesadmin.fire.containmentdatetime'
fieldmap['guid'] = 'firesadmin.fire.irwinid'
fieldmap['totalcost'] = 'firesadmin.fire.estimatedtotalcost'
fieldmap['cause'] = 'firesadmin.fire.generalcause'
fieldmap['id'] = 'firesadmin.fire.id'

In [12]:
len(filelist)

18

In [13]:
def getfiredf(filepath):
    data = blm.get_raw_fire_data(filepath, tryremote=False)
    fieldnames = blm.sanitize_dataline(data[0], blanks=3, lower=True)
    rawdata = [blm.sanitize_dataline(
                item.strip(), blanks=2, lower=False) for item in data[1::2]]

    # the transformed data will be a list of dictionaries
    data = blm.get_firedata(fieldnames, rawdata)
    firedf = pd.DataFrame(data, columns=list(fieldmap.values()))
    firedf.columns = [list(fieldmap.keys())]
    firedf = cleanup_df(firedf)
    firedf['id'] = firedf['id'].astype('int')
    firedf.set_index('id', inplace=True)
    return firedf

In [14]:
def mergefireupdate(basefiredf, newfiredf):
    now = dt.datetime.now()
    for index, row in newfiredf.iterrows():
        try: 
            comparecond = (~(
                basefiredf.loc[index, ['acreage', 'maintext']] == newfiredf.loc[index, ['acreage', 'maintext']])).any()
            if comparecond:
                basefiredf.loc[index, 'prev_acreage'] = basefiredf.loc[index, 'acreage'] 
                basefiredf.loc[index, 'prev_text'] = basefiredf.loc[index, 'maintext'] 
                basefiredf.loc[index, basefields] = newfiredf.loc[index, basefields]
                basefiredf.loc[index, 'updatecount'] += 1
        except KeyError:
                basefiredf.loc[index] = newfiredf.loc[index]
                basefiredf.loc[index, 'updatecount'] = 1
                basefiredf.loc[index, 'prev_acreage'] = np.nan
                basefiredf.loc[index, 'prev_text'] = ''
                basefiredf.loc[index, 'current'] = True
        if ((now - basefiredf.loc[index, 'lastupdatedatetime']).days > 8):
            basefiredf.loc[index, 'current'] = False
    return basefiredf
            

In [15]:
try:
    firedf = pd.read_pickle(LASTFIREDATA)
    startidx = 0
except FileNotFoundError:
    firedf = getfiredf(filelist[0])
    firedf['prev_acreage'] = np.nan
    firedf['prev_text'] = ''
    firedf['current'] = True
    firedf['updatecount'] = 1
    startidx = 1

In [16]:
basefields = list(fieldmap.keys())
basefields.remove('id')

In [17]:
for fn in filelist[startidx:]:
    print("Opening {}".format(fn))
    try:
        newfiredf = getfiredf(fn)
        newfiredf['prev_acreage'] = np.nan
        newfiredf['prev_text'] = ''
        newfiredf['current'] = True
        newfiredf['updatecount'] = 1
    except IOError:
        continue
    except IndexError:
        continue
    firedf0 = mergefireupdate(firedf, newfiredf)

Opening archive/20170726_025909_blmfiredata.txt
Opening archive/20170726_035911_blmfiredata.txt
Opening archive/20170726_045910_blmfiredata.txt
Opening archive/20170726_055909_blmfiredata.txt
Opening archive/20170726_065912_blmfiredata.txt
Opening archive/20170726_135908_blmfiredata.txt
Opening archive/20170726_145909_blmfiredata.txt
Opening archive/20170726_155908_blmfiredata.txt
Opening archive/20170726_165911_blmfiredata.txt
Opening archive/20170726_175909_blmfiredata.txt
Opening archive/20170726_185907_blmfiredata.txt
Opening archive/20170726_195906_blmfiredata.txt
Opening archive/20170726_205906_blmfiredata.txt
Opening archive/20170726_215905_blmfiredata.txt
Opening archive/20170726_225906_blmfiredata.txt
Opening archive/20170726_235907_blmfiredata.txt
Opening archive/20170727_005907_blmfiredata.txt
Opening archive/20170727_015907_blmfiredata.txt


In [18]:
firedf[firedf['current']][[
    'designation', 'cause', 'acreage', 'prev_acreage', 
    'lastupdatedatetime', 'maintext', 'prev_text', 'longitude', 'latitude']].to_dict('records')

[{'acreage': 0.1,
  'cause': 'Human',
  'designation': 'Salchaket Subdivision',
  'lastupdatedatetime': NaT,
  'latitude': '64.39848',
  'longitude': '-146.777772',
  'maintext': '',
  'prev_acreage': nan,
  'prev_text': ''},
 {'acreage': 5.0,
  'cause': 'Lightning',
  'designation': 'North Ladue River',
  'lastupdatedatetime': NaT,
  'latitude': '63.525883',
  'longitude': '-140.87755',
  'maintext': '',
  'prev_acreage': nan,
  'prev_text': ''},
 {'acreage': 0.1,
  'cause': 'Human',
  'designation': 'Crooked Creek',
  'lastupdatedatetime': NaT,
  'latitude': '61.870333',
  'longitude': '-158.105167',
  'maintext': '',
  'prev_acreage': nan,
  'prev_text': ''},
 {'acreage': 0.1,
  'cause': 'Human',
  'designation': 'Boulder Creek',
  'lastupdatedatetime': NaT,
  'latitude': '64.9',
  'longitude': '-144.95',
  'maintext': '',
  'prev_acreage': nan,
  'prev_text': ''},
 {'acreage': 0.1,
  'cause': 'Human',
  'designation': 'Parks Pullout',
  'lastupdatedatetime': NaT,
  'latitude': '64.

In [19]:
def savecurrentstate(filelist, firedf0, outfile, outdataframe):
    with open(outfile, 'w') as dest:
        for line in oldfilelist:
            print(line, file=dest)
        for line in filelist:
            print(line, file=dest)
    firedf0.to_pickle(outdataframe)

In [20]:
savecurrentstate(filelist, firedf, 'aiccfilelist.txt', 'firedata.pickle')

In [21]:
sqkm(113, .5)

NameError: name 'sqkm' is not defined

In [22]:
firedf[firedf['current']][[
    'designation', 'cause', 'acreage', 'prev_acreage', 
    'lastupdatedatetime', 'discoverydatetime', 'maintext', 'prev_text', 'longitude', 'latitude']].sort_values(
    by='lastupdatedatetime', ascending=False).to_dict('records')

[{'acreage': 0.1,
  'cause': 'Human',
  'designation': 'Jakolof Bay',
  'discoverydatetime': Timestamp('2017-07-24 09:26:38.415000'),
  'lastupdatedatetime': Timestamp('2017-07-26 15:48:45.415000'),
  'latitude': '59.463267',
  'longitude': '-151.534567',
  'maintext': 'Fire occurred on 7/24 and Seldovia VFD responded,  fire was reported to be 8`x8` smoldering in grass and duff.  Fire was contained and controlled by VFD and is now in monitor status.',
  'prev_acreage': 0.1,
  'prev_text': ''},
 {'acreage': 0.1,
  'cause': 'Human',
  'designation': 'Grand Ave',
  'discoverydatetime': Timestamp('2017-07-26 13:32:38.521000'),
  'lastupdatedatetime': Timestamp('2017-07-26 15:29:42.521000'),
  'latitude': '61.713533',
  'longitude': '-148.889683',
  'maintext': '911 reported a structure fire with possible wildland involvement.  On location Forestry located minimal wildland in fire.  Engine crew members suppressed the fire and placed it on monitor status.',
  'prev_acreage': nan,
  'prev_tex

In [23]:
import math
def sqkm(acres, perc):
    return math.sqrt(acres/2.5/100) * perc * 3.14 * 2 * 0.02

In [52]:
sqkm(2661, .8)

0.32781744367498206